# Modules

In [ ]:
from tweepy import API
from tweepy import OAuth1UserHandler
from tweepy import Stream
import tweepy
import time
import pyarrow
import pandas as pd
pd.set_option('display.max_rows', 500)

# 1. Twitter Auth and API

Keys are not shown

# 2. Create blank lists in a dict

In [ ]:
dmo_twitter = pd.read_csv('dmo_twitter.csv')

In [ ]:
display(dmo_twitter)

In [ ]:
dmo_twitter_filtered = dmo_twitter[dmo_twitter['twitter_handle'].notnull()]
dmo_twitter_filtered = dmo_twitter_filtered[dmo_twitter_filtered['exclude'] != 'yes']
dmo_twitter_filtered['country'] = dmo_twitter_filtered['country_name'].replace(" ", "_",regex=True)
dmo_twitter_filtered['country'] = dmo_twitter_filtered['country'].str.lower()
dmo_twitter_filtered.reset_index()
display(dmo_twitter_filtered)

In [ ]:
dmo_dict = {key:[] for key in dmo_twitter_filtered['country']}
display(dmo_dict)

# 3. Loop collect tweets

In [ ]:
for key in without_keys(dmo_dict, completed_dmo):
    twitter_handle = dmo_twitter_filtered.loc[dmo_twitter_filtered["country"] == key].iloc[0]['twitter_handle']
    print(twitter_handle)
    query_words = ["from:",twitter_handle," -is:retweet -is:reply -is:quote lang:en"] #Alternatively, -is:quote can be removed to collect all quotes here
    query = "".join(query_words)
    country_tweet = []
    
    for tweet in tweepy.Paginator(client.search_all_tweets,query = query, 
                                                       start_time = "2017-01-01T00:00:00Z", 
                                                       end_time = "2022-09-30T23:59:59Z", 
                                                       max_results=500,
                                                       tweet_fields=["id", "author_id",  "created_at", "text", "source", "lang", "in_reply_to_user_id", "conversation_id", "public_metrics", "referenced_tweets", "reply_settings"],
                                                       user_fields=["name", "username", "location", "verified", "description", "created_at"],
                                                       place_fields=["full_name", "id", "country", "country_code", "geo", "name", "place_type"]).flatten(limit = 100000):
        tweet_page = tweet.data
        country_tweet.append(tweet_page)
        
    country_df = pd.DataFrame.from_records(country_tweet)
    country_df_metrics = pd.json_normalize(country_df['public_metrics'])
    dmo_dict[key] = pd.concat([country_df.drop(columns = ['public_metrics']), country_df_metrics], axis =1)
    dmo_dict[key]['country'] = key
    

# 4. Merge DF in Dict

In [ ]:
dmo_df = pd.concat(dmo_dict.values()) # Add your sub_df one by one

print(dmo_df)

# 5. Create blank lists in a dict (replies)

In [ ]:
reply_dict = {key:[] for key in dmo_twitter_filtered['country']}
display(reply_dict)

# 6. Loop collect replies

In [ ]:
for key in without_keys(dmo_dict, completed_dmo): #dmo_dict.keys(): 
    twitter_handle = dmo_twitter_filtered.loc[dmo_twitter_filtered["country"] == key].iloc[0]['twitter_handle']
    print(twitter_handle)
    query_words = ["from:",twitter_handle," -is:retweet lang:en (is:reply OR is:quote)"] #Alternatively, is:quote can be removed to exclude quotes
    query = "".join(query_words)
    country_reply = []
    
    for tweet in tweepy.Paginator(client.search_all_tweets,query = query, 
                                                       start_time = "2017-01-01T00:00:00Z", 
                                                       end_time = "2022-09-30T23:59:59Z", 
                                                       max_results=500,
                                                       tweet_fields=["id", "author_id",  "created_at", "text", "source", "lang", "in_reply_to_user_id", "conversation_id", "public_metrics", "referenced_tweets", "reply_settings"],
                                                       user_fields=["name", "username", "location", "verified", "description", "created_at"],
                                                       place_fields=["full_name", "id", "country", "country_code", "geo", "name", "place_type"]).flatten(limit = 100000):
        tweet_page = tweet.data
        country_reply.append(tweet_page)
        
    reply_df = pd.DataFrame.from_records(country_reply)
    try:
        reply_df_metrics = pd.json_normalize(reply_df['public_metrics'])
        reply_dict[key] = pd.concat([reply_df.drop(columns = ['public_metrics']), reply_df_metrics], axis =1)
        reply_dict[key]['country'] = key
    except Exception:
        pass
    

In [ ]:
dmo_replies_df = pd.concat(reply_dict.values()) # Add your sub_df one by one

print(dmo_replies_df)

# 7. Export to Feather

In [ ]:
dmo_df.reset_index().to_feather("dmo_tweets.feather")

In [ ]:
dmo_replies_df.reset_index().to_feather("dmo_replies.feather")